# Assignment 2: recommender System

## Code from the lecture

In [2]:
#read data to DataFrames
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../../DATA/movielens100k/u.user', sep='|', names=u_cols, encoding = "ISO-8859-1")

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../../DATA/movielens100k/u.data', sep='\t', names=r_cols, encoding = "ISO-8859-1")

m_cols=['movie_id', 'title', 'release date', 'video release date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../../DATA/movielens100k/u.item', sep='|', names=m_cols ,encoding = "ISO-8859-1" )

FileNotFoundError: [Errno 2] No such file or directory: '../../DATA/movielens100k/u.user'

In [ ]:
#generate matrix (this can be done more efficiently!)
R=np.zeros((np.shape(users)[0],np.shape(movies)[0]))
for i in range(np.shape(ratings)[0]):
    R[ratings['user_id'][i]-1, ratings['movie_id'][i]-1]=ratings['rating'][i]

In [ ]:
def CosineDist(a,b):
    result = a.dot(b)
    var = np.linalg.norm(a)*np.linalg.norm(b)
    if var > 0:
        return result/var
    else:
        return result

In [ ]:
#again, implementation not efficient: better use lamda opperators 
D=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D[i,j]=CosineDist(R[:,i],R[:,j])

In [ ]:
def getTopN(movie_id,D,N=5):
    return D[movie_id,:].argsort()[-N:]

def getIDbyName(name):
    if np.size(movies.movie_id[movies.title.str.contains(name)]) > 0:
        m = int(movies.movie_id[movies.title.str.contains(name)][:1]), str(movies.title[movies.title.str.contains(name)][:1])
        return m[0]-1
    else:
        return -1
    
def getNameByID(IDs):
    res=movies.iloc[IDs]
    return res.title

def CII(title):
    if getIDbyName(title) > 0:
        print ("recommending movies for: '" + str(getNameByID(getIDbyName(title)))+"'")
        return getNameByID(getTopN(getIDbyName(title)))[::-1]
    else:
        print ("no movie title containing " + str(title) + "found...")
        
def Score_byID(ID, D, Test):
    #print ("Hit Scores for: ", getNameByID(ID))
    res_id = getTopN(ID,D)[::-1]
    res_title = getNameByID(getTopN(ID,D))[::-1]
    res_score = Test[res_id]
    return res_id, res_title, res_score, np.mean(res_score)
    
#compute scores for all test users
def test_Score(D_train, R_test):
    userScores=[]
    for i in range(R_test.shape[0]):
        userScore=0
        userTop = np.argsort(R_test[i])[-5:]
        for e in userTop:
            res_id, res_title, res_score, av_score = Score_byID(e,D_train, R_test[i])
            userScore+=av_score
        userScores.append(userScore/(5))
    return userScores

## 4.1 Alternative Distance Measures
Implement a different distance measure, e.g. eucledian and compare the ressults.

In [ ]:
def eucDistance(a,b):
    return np.linalg.norm(b-a)

eD = np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            eD[i,j]=eucDistance(R[:,i],R[:,j])
np.save("movie_dist",eD)

In [ ]:
#import matplotlib.pyplot as plt

fig=plt.figure(figsize=(20, 20))
ax1=fig.add_subplot(1, 3, 1)
plt.xlabel('movie id')
plt.ylabel('user id ')
ax1.title.set_text('Cosine distance')
plt.imshow(D)
ax2= fig.add_subplot(1, 3, 2)
plt.xlabel('movie id')
plt.ylabel('user id ')
ax2.title.set_text('Eucledian distance')
plt.imshow(eD)
plt.show()

### Compare results:

In [ ]:
#split train and test data
from sklearn.model_selection import train_test_split
R_train, R_test = train_test_split(R, test_size=0.1)

In [ ]:
#cos distance
cos_dis=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            cos_dis[i,j]=CosineDist(R_train[:,i],R_train[:,j])
            
np.save("movie_cos_dis",cos_dis)

In [ ]:
#eucledian distance
euc_dis=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            euc_dis[i,j]=eucDistance(R_train[:,i],R_train[:,j])
            
np.save("movie_euc_dis",euc_dis)

In [ ]:
test_cos=test_Score(cos_dis,R_test)
test_eD=test_Score(euc_dis,R_test)

print('Mean scores of cosine dist: ', np.mean(test_cos))
print('Mean scores of euclidean dist: ', np.mean(test_eD))

In [ ]:
fig=plt.figure(figsize=(17, 5))
ax1=fig.add_subplot(1, 3, 1)
ax1.title.set_text('Eucledian')
plt.hist(test_eD)
ax2= fig.add_subplot(1, 3, 2)
ax2.title.set_text('Cos')
plt.hist(test_cos)
plt.show()

## Result:
Cosine distance measure is better than the eucledian distance.

## 4.2 Baseline Results
Implement and compare results of
* random recommendations
* allways recommending the top 5 movies (over all users)

### Random recommendations

In [ ]:
#random recommendations - methods

def RandomScore(ID, D, Test): #score of random id
    m_id = np.array([np.random.randint(np.shape(D)[0]) for b in range(5)])
    title = [getNameByID(id) for id in m_id]
    score = Test[m_id]
    return m_id, title, score, np.mean(score)

def testRandScore(D_train, R_test): #all test users
    Scores=[]
    for i in range(R_test.shape[0]):
        Score=0
        Top = np.argsort(R_test[i])[-5:]
        for x in Top:
            res_id, res_title, res_score, av_score = RandomScore(x,D_train, R_test[i])
            Score+=av_score
        Scores.append(Score/(5))
    return Scores

In [ ]:
# random recommendations - test
test_rand = testRandScore(cos_dis, R_test)
print('mean score of random cosine:',np.mean(test_rand))



### top 5 movies

In [ ]:
# recommending the top 5 movies over all users - methods

def TopMovies(ID, D):
    x = np.sum(D,axis=0)
    return np.argsort(x)[-5:]

def Score(ID, D, Test):
    res_id = TopMovies(ID,D)[::-1]
    res_title = getNameByID(TopMovies(ID,D))[::-1]
    res_score = Test[res_id]
    return res_id, res_title, res_score, np.mean(res_score)

def TestTop5(D_train, R_test):
    Scores=[]
    for i in range(R_test.shape[0]):
        ScoreUser=0
        Top = np.argsort(R_test[i])[-5:]
        for e in Top:
            res_id, res_title, res_score, av_score = Score(e,D_train, R_test[i])
            ScoreUser+=av_score
        Scores.append(ScoreUser/(5))
    return Scores

In [ ]:
# recommending the top 5 movies over all users - test
test_top=TestTop5(cos_dis, R_test)
print(test_top)
print('mean score of top 5 cosine:',np.mean(test_top))

In [ ]:
fig=plt.figure(figsize=(17, 5))
ax1=fig.add_subplot(1, 3, 1)
ax1.title.set_text('Cos')
plt.hist(test_cos)
ax2=fig.add_subplot(1, 3, 2)
ax2.title.set_text('Top5')
plt.hist(test_top)
ax3= fig.add_subplot(1, 3, 3)
ax3.title.set_text('Random')
plt.hist(test_rand)

plt.show()

## 4.3 SVD
Implement the SVD approach. Hints: 
* use numpy.linalg.svd()
* apply SVD on $D$
* plot the singular values and decide where to cut off the reconstruction
* reconstruct and evaluate $D'$


In [ ]:
U,S,V = np.linalg.svd(D)
D.shape, U.shape, S.shape, V.shape

In [ ]:
plt.plot(S)
plt.show()

In [ ]:
np.allclose(D, np.dot(U * S, V[:1682,:])) 
cut = 100
d_approx = np.dot(U[:,:cut] * S[:cut], V[:cut,:])

In [ ]:
import scipy.spatial

D_app=scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(d_approx.T, metric='cosine'))
D_app=np.abs(np.nan_to_num(D_app-1))
np.fill_diagonal(D_app,0)

In [ ]:
test_app= test_Score(D_app,R_test)

In [ ]:
print("Mean score cos:",np.mean(test_cos), "Mean score approx cos:",np.mean(test_app))

In [ ]:
fig=plt.figure(figsize=(17, 5))
ax1=fig.add_subplot(1, 2, 1)
ax1.title.set_text('Cos')
plt.hist(test_cos)
ax2= fig.add_subplot(1, 2, 2)
ax2.title.set_text('Cos approx')
plt.hist(test_app)
plt.show()